In [82]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Import the data set

In [83]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data Cleaning

In [11]:
lab_data = pd.read_csv('../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data cleaning
erroneous = lab_data[lab_data['Thorax_length'] == '.']
erroneous

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
253,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,6,3,female,.,1.65,0.573,1.415,1.988,1.986,0.898,1.207,1.003,.


In [12]:
similar_rows = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Gogango_Creek')
    & (lab_data['Sex'] == 'female')
    & (lab_data['Thorax_length'] != ".")
]
similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])
similar_rows.head()

/var/folders/j_/9f1md2vd0_s6kpn12szf1t2c0000gn/T/ipykernel_67286/3776846164.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])


,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
162,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,1,female,1.166,1.910,0.578,1.675,2.254,2.252,0.982,1.381,1.123,1.932
164,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,2,female,1.174,1.829,0.622,1.549,2.170,2.170,0.997,1.360,1.117,1.848
166,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,3,female,1.199,1.864,0.585,1.659,2.244,2.242,1.009,1.376,1.142,1.870
167,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,1,female,1.186,1.872,0.628,1.571,2.199,2.199,1.002,1.356,1.143,1.854
169,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,2,female,1.176,1.872,0.630,1.579,2.207,2.207,0.966,1.345,1.103,1.876


In [13]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

# Model Building

In [84]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [85]:
lab_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1730 entries, 0 to 1730
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Species        1730 non-null   object 
 1   Population     1730 non-null   object 
 2   Latitude       1730 non-null   float64
 3   Longitude      1730 non-null   float64
 4   Year_start     1730 non-null   int64  
 5   Year_end       1730 non-null   int64  
 6   Temperature    1730 non-null   int64  
 7   Vial           1730 non-null   int64  
 8   Replicate      1730 non-null   int64  
 9   Sex            1730 non-null   object 
 10  Thorax_length  1730 non-null   float64
 11  l2             1730 non-null   float64
 12  l3p            1730 non-null   float64
 13  l3d            1730 non-null   float64
 14  lpd            1730 non-null   float64
 15  l3             1730 non-null   float64
 16  w1             1730 non-null   float64
 17  w2             1730 non-null   float64
 18  w3           

In [86]:
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
0,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,female,1.238,2.017,0.659,1.711,2.370,2.370,1.032,1.441,1.192,1.914
1,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,male,1.113,1.811,0.609,1.539,2.148,2.146,0.938,1.299,1.066,1.928
2,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,female,1.215,1.985,0.648,1.671,2.319,2.319,0.991,1.396,1.142,1.908
3,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,male,1.123,1.713,0.596,1.495,2.091,2.088,0.958,1.286,1.062,1.860
4,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,2,1,female,1.218,1.938,0.641,1.658,2.298,2.298,1.010,1.418,1.148,1.886


In [87]:
#TODO: Currently using decision tree so no need to normalise, but must do for other methods

In [88]:
lab_data = shuffle(lab_data)
lab_data['class'] = lab_data.apply(
    lambda x: f"{x['Species']}, {x['Sex']}",
    axis=1
)
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class
1119,D._buzzatii,Gogango_Creek,-23.77,150.17,1994,1994,25,7,3,female,...,1.889,0.624,1.563,2.186,2.185,0.943,1.344,1.071,1.835,"D._buzzatii, female"
334,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,30,10,1,male,...,1.450,0.508,1.257,1.765,1.764,0.855,1.121,0.938,1.707,"D._aldrichi, male"
936,D._buzzatii,Binjour,-25.52,151.45,1994,1994,25,7,1,male,...,1.676,0.552,1.459,2.010,2.009,0.912,1.246,1.021,1.761,"D._buzzatii, male"
250,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,6,1,male,...,1.554,0.543,1.314,1.856,1.855,0.863,1.129,0.986,1.773,"D._aldrichi, male"
183,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,4,3,female,...,1.843,0.654,1.527,2.182,2.181,1.000,1.339,1.145,1.886,"D._aldrichi, female"


In [89]:

train, test = train_test_split(lab_data, test_size=0.3)

In [90]:
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=5)
#TODO: Add Thorax Length and wing_loading to this training
classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [91]:
train['class_predictions'] = classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [92]:
train.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class,class_predictions
242,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,4,3,male,...,0.563,1.376,1.939,1.937,0.902,1.209,1.018,1.801,"D._aldrichi, male","D._buzzatii, male"
1145,D._buzzatii,Gogango_Creek,-23.77,150.17,1994,1994,30,2,1,male,...,0.522,1.392,1.914,1.913,0.849,1.162,0.953,1.748,"D._buzzatii, male","D._aldrichi, female"
814,D._aldrichi,Wahruna,-25.20,151.17,1994,1994,30,6,1,female,...,0.522,1.366,1.888,1.886,0.844,1.136,0.954,1.787,"D._aldrichi, female","D._buzzatii, male"
1443,D._buzzatii,Oxford_Downs,-21.77,148.85,1994,1994,25,2,3,female,...,0.639,1.474,2.113,2.112,0.945,1.311,1.061,1.926,"D._buzzatii, female","D._buzzatii, male"
1716,D._buzzatii,Wahruna,-25.20,151.17,1994,1994,30,8,2,male,...,0.506,1.288,1.795,1.793,0.785,1.086,0.913,1.788,"D._buzzatii, male","D._buzzatii, male"


In [93]:
#Correct predictions
train.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()

687

In [94]:
#Incorrect predictions
train.apply(
    lambda x: 1 if x['class_predictions'] != x['class'] else 0,
    axis=1
).sum()

524

In [95]:
# Retry with a deeper depth

In [96]:
deep_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=10)

deep_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [97]:
train['deep_class_predictions'] = deep_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [98]:
#Correct deep predictions
train.apply(
    lambda x: 1 if x['deep_class_predictions'] == x['class'] else 0,
    axis=1
).sum()

971

In [99]:
min_leaf_classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

min_leaf_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

In [100]:
train['min_leaf_predictions'] = min_leaf_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [101]:
#Correct min_leaf predictions
train.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

1034

In [102]:
fully_fit_classifier = DecisionTreeClassifier(criterion='entropy')

fully_fit_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

DecisionTreeClassifier(criterion='entropy')

In [103]:
train['fully_fit_predictions'] = fully_fit_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [104]:
#Correct fully_fit predictions
train.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

1211

In [105]:
# Checking the min leaf and fully fit classifiers against the test set

In [106]:
#Correst min_leaf predictions on the test set

In [107]:
test['min_leaf_predictions'] = min_leaf_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['fully_fit_predictions'] = fully_fit_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])


correct_min_leaf_classification_test = test.apply(
    lambda x: 1 if x['min_leaf_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Min leaf correct predictions test set:" + str(correct_min_leaf_classification_test))

correct_fully_fit_classification_test = test.apply(
    lambda x: 1 if x['fully_fit_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Fully fit predictions test set:" + str(correct_fully_fit_classification_test))


Min leaf correct predictions test set:264
Fully fit predictions test set:266


In [108]:
print(f"accuracy: {correct_min_leaf_classification_test / len(test)}")

accuracy: 0.5086705202312138
